In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.optimize import minimize

# Disable interactive mode
plt.ioff()

# Define paths
ice_folder_path = r"C:\Users\yliu117\Desktop\EV\ACC_data"
ev_folder_path = r"C:\Users\yliu117\Desktop\EV\EV-ACC data\combined"
new_data_path = r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025"
output_base_path = r"C:\Users\yliu117\Desktop\EV_vs_ICE"

# Initial IDM parameters for ICE vehicles
ice_initial_params = {
    "A": {"Min": {"v0": 43.60, "tau": 1.00, "s0": 8.00, "delta": 13.50, "alpha": 0.90, "beta": 9.00},
          "Max": {"v0": 44.10, "tau": 2.20, "s0": 6.30, "delta": 15.50, "alpha": 0.60, "beta": 5.20}},
    "B": {"Min": {"v0": 42.80, "tau": 1.00, "s0": 8.00, "delta": 16.20, "alpha": 0.80, "beta": 9.00},
          "Max": {"v0": 42.40, "tau": 2.30, "s0": 7.90, "delta": 17.50, "alpha": 0.70, "beta": 8.90}},
    "C": {"Min": {"v0": 43.70, "tau": 1.10, "s0": 7.80, "delta": 17.10, "alpha": 0.60, "beta": 8.60},
          "Max": {"v0": 43.60, "tau": 2.30, "s0": 8.00, "delta": 17.10, "alpha": 0.60, "beta": 6.50}},
    "D": {"Min": {"v0": 43.40, "tau": 1.00, "s0": 8.00, "delta": 14.80, "alpha": 0.60, "beta": 9.00},
          "Max": {"v0": 44.70, "tau": 2.20, "s0": 8.00, "delta": 17.30, "alpha": 0.70, "beta": 8.70}},
    "E": {"Min": {"v0": 42.60, "tau": 1.30, "s0": 4.20, "delta": 18.90, "alpha": 0.90, "beta": 9.00},
          "Max": {"v0": 44.80, "tau": 2.00, "s0": 8.00, "delta": 19.60, "alpha": 1.30, "beta": 9.00}},
    "F": {"Min": {"v0": 44.70, "tau": 1.00, "s0": 8.00, "delta": 19.70, "alpha": 0.80, "beta": 9.00},
          "Max": {"v0": 41.90, "tau": 1.90, "s0": 7.10, "delta": 13.60, "alpha": 0.80, "beta": 9.00}},
    "G": {"Min": {"v0": 44.90, "tau": 1.00, "s0": 8.00, "delta": 19.80, "alpha": 0.80, "beta": 9.00},
          "Max": {"v0": 44.00, "tau": 2.00, "s0": 4.10, "delta": 11.10, "alpha": 0.90, "beta": 9.00}}
}

# Initial IDM parameters for EV vehicles
ev_initial_params = {
    "Short": {"v0": 33.37, "tau": 1.56, "s0": 2.04, "delta": 3.99, "alpha": 2.06, "beta": 9.00},
    "Medium": {"v0": 33.34, "tau": 1.63, "s0": 2.02, "delta": 4.02, "alpha": 2.01, "beta": 8.97},
    "Long": {"v0": 33.36, "tau": 1.67, "s0": 2.41, "delta": 3.96, "alpha": 1.86, "beta": 8.96},
    "Xlong": {"v0": 33.30, "tau": 2.17, "s0": 5.23, "delta": 3.66, "alpha": 1.65, "beta": 8.98},
    "Polestar": {"v0": 33.37, "tau": 1.56, "s0": 2.04, "delta": 3.99, "alpha": 2.06, "beta": 9.00},
    "Tesla": {"v0": 33.37, "tau": 1.56, "s0": 2.04, "delta": 3.99, "alpha": 2.06, "beta": 9.00}
}

# Parameter bounds
ice_bounds = {
    'v0': (30.0, 50.0),
    'tau': (0.5, 3.0),
    's0': (2.0, 10.0),
    'delta': (5.0, 20.0),
    'alpha': (0.5, 2.0),
    'beta': (5.0, 10.0)
}

ev_bounds = {
    'v0': (20, 40),
    'tau': (0.5, 3.0),
    's0': (1.0, 10.0),
    'delta': (2.0, 10.0),
    'alpha': (0.5, 3.0),
    'beta': (5.0, 10.0)
}

# Plot parameters
plot_params = {
    'xlabel_fontsize': 10,
    'ylabel_fontsize': 10,
    'legend_fontsize': 7,
    'tick_labelsize': 8,
    'grid_linestyle': '--',
    'grid_alpha': 0.7,
    'title_fontsize': 7
}

# Color maps
ice_color_map = {
    "dip_min": "blue",
    "osc_min": "purple",
    "stepH_min": "cyan",
    "stepL_min": "brown",
    "dip_max": "red",
    "osc_max": "green",
    "stepH_max": "orange",
    "stepL_max": "magenta"
}

ev_color_map = {
    "35": sns.husl_palette(3)[0],
    "45": sns.husl_palette(3)[1],
    "55": sns.husl_palette(3)[2]
}

# IDM model function
def idm_acceleration(v, delta_v, s, v0, tau, s0, delta, alpha, beta):
    s_star = s0 + v * tau + (v * delta_v) / (2 * np.sqrt(alpha * beta))
    s_star = max(s_star, s0)
    acc = alpha * (1 - (v / v0) ** delta - (s_star / s) ** 2)
    return acc

# Simulate IDM trajectory
def simulate_idm(lead_speed, lead_time, params, initial_v, initial_s, dt):
    v0 = params['v0']
    tau = params['tau']
    s0 = params['s0']
    delta = params['delta']
    alpha = params['alpha']
    beta = params['beta']
    
    v = [initial_v]
    s = [initial_s]
    t = [lead_time[0]]
    
    for i in range(1, len(lead_time)):
        delta_v = v[-1] - lead_speed[i]
        acc = idm_acceleration(v[-1], delta_v, s[-1], v0, tau, s0, delta, alpha, beta)
        v_new = v[-1] + acc * dt
        v_new = max(v_new, 0)
        s_new = s[-1] + (lead_speed[i] - v_new) * dt
        s_new = max(s_new, s0)
        v.append(v_new)
        s.append(s_new)
        t.append(t[-1] + dt)
    
    return np.array(t), np.array(v), np.array(s)

# Calculate RMSE
def calculate_rmse(actual, predicted):
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted)
    if np.sum(valid_mask) == 0:
        return np.nan
    return np.sqrt(np.mean((actual[valid_mask] - predicted[valid_mask]) ** 2))

# Objective function for optimization
def objective_function(params, datasets, dt):
    v0, tau, s0, delta, alpha, beta = params
    idm_params = {'v0': v0, 'tau': tau, 's0': s0, 'delta': delta, 'alpha': alpha, 'beta': beta}
    total_rmse = 0.0
    
    for time, lead_speed, follow_speed, spacing in datasets:
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else s0
        idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, idm_params, initial_v, initial_s, dt)
        spacing_rmse = calculate_rmse(spacing, idm_spacing)
        if not np.isnan(spacing_rmse):
            total_rmse += spacing_rmse
    
    return total_rmse

# Create output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = os.path.join(output_base_path, f"EV_ICE_IDM_Calibration_{timestamp}")
os.makedirs(output_folder, exist_ok=True)

# ICE vehicles and conditions
ice_vehicles = ["A", "B", "C", "D", "E", "F", "G"]
ice_conditions = {
    "Min": ["dip_min", "osc_min", "stepH_min", "stepL_min"],
    "Max": ["dip_max", "osc_max", "stepH_max", "stepL_max"]
}

# EV vehicles and conditions
ev_vehicles = ["Short", "Medium", "Long", "Xlong"]
ev_conditions = ["35", "45", "55"]

# New datasets for EV
new_groups = {
    "Polestar": [
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\4.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\5.csv"
    ],
    "Tesla": [
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\35\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\45\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\45\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\5_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\5_desired\55\35\2.csv"
    ]
}

# Store best parameters and RMSEs
ice_best_params = {}
ice_rmse_results = {}
ev_best_params = {}
ev_rmse_results = {}

# Optimize parameters for ICE vehicles
for vehicle in ice_vehicles:
    for setting in ["Min", "Max"]:
        datasets = []
        for condition in ice_conditions[setting]:
            csv_file = f"Veh{vehicle}_{condition}.csv"
            try:
                df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
                time = df.iloc[:, 0].values
                lead_speed = df.iloc[:, 2].values
                follow_speed = df.iloc[:, 1].values
                spacing = df.iloc[:, 3].values
                datasets.append((time, lead_speed, follow_speed, spacing))
            except Exception as e:
                print(f"Error reading {csv_file}: {e}")
                continue
        
        initial_guess = [
            ice_initial_params[vehicle][setting]['v0'],
            ice_initial_params[vehicle][setting]['tau'],
            ice_initial_params[vehicle][setting]['s0'],
            ice_initial_params[vehicle][setting]['delta'],
            ice_initial_params[vehicle][setting]['alpha'],
            ice_initial_params[vehicle][setting]['beta']
        ]
        
        param_bounds = [
            ice_bounds['v0'],
            ice_bounds['tau'],
            ice_bounds['s0'],
            ice_bounds['delta'],
            ice_bounds['alpha'],
            ice_bounds['beta']
        ]
        
        result = minimize(
            objective_function,
            initial_guess,
            args=(datasets, 0.1),
            method='SLSQP',
            bounds=param_bounds,
            options={'disp': True, 'maxiter': 1000}
        )
        
        ice_best_params[(vehicle, setting)] = {
            'v0': result.x[0],
            'tau': result.x[1],
            's0': result.x[2],
            'delta': result.x[3],
            'alpha': result.x[4],
            'beta': result.x[5]
        }
        
        ice_rmse_results[(vehicle, setting)] = {}
        for condition, (time, lead_speed, follow_speed, spacing) in zip(ice_conditions[setting], datasets):
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
            initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ice_best_params[(vehicle, setting)]['s0']
            idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ice_best_params[(vehicle, setting)], initial_v, initial_s, 0.1)
            speed_rmse = calculate_rmse(follow_speed, idm_speed)
            spacing_rmse = calculate_rmse(spacing, idm_spacing)
            ice_rmse_results[(vehicle, setting)][condition] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Optimize parameters for EV vehicles
for vehicle in ev_vehicles:
    datasets = []
    for condition in ev_conditions:
        csv_file = f"{vehicle}_{condition}.csv"
        try:
            df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778
            follow_speed = df['Smooth Speed Follower'].values * 0.27778
            spacing = df['Spacing'].values
            datasets.append((time, lead_speed, follow_speed, spacing))
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
            continue
    
    initial_guess = [
        ev_initial_params[vehicle]['v0'],
        ev_initial_params[vehicle]['tau'],
        ev_initial_params[vehicle]['s0'],
        ev_initial_params[vehicle]['delta'],
        ev_initial_params[vehicle]['alpha'],
        ev_initial_params[vehicle]['beta']
    ]
    
    param_bounds = [
        ev_bounds['v0'],
        ev_bounds['tau'],
        ev_bounds['s0'],
        ev_bounds['delta'],
        ev_bounds['alpha'],
        ev_bounds['beta']
    ]
    
    result = minimize(
        objective_function,
        initial_guess,
        args=(datasets, 0.02),
        method='SLSQP',
        bounds=param_bounds,
        options={'disp': True, 'maxiter': 1000}
    )
    
    ev_best_params[vehicle] = {
        'v0': result.x[0],
        'tau': result.x[1],
        's0': result.x[2],
        'delta': result.x[3],
        'alpha': result.x[4],
        'beta': result.x[5]
    }
    
    ev_rmse_results[vehicle] = {}
    for condition, (time, lead_speed, follow_speed, spacing) in zip(ev_conditions, datasets):
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ev_best_params[vehicle]['s0']
        idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ev_best_params[vehicle], initial_v, initial_s, 0.02)
        speed_rmse = calculate_rmse(follow_speed, idm_speed)
        spacing_rmse = calculate_rmse(spacing, idm_spacing)
        ev_rmse_results[vehicle][condition] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Optimize parameters for new EV groups (Polestar and Tesla)
for group in new_groups:
    datasets = []
    for file_path in new_groups[group]:
        try:
            df = pd.read_csv(file_path)
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778
            follow_speed = df['Smooth Speed Follower'].values * 0.27778
            spacing = df['Spacing'].values
            datasets.append((time, lead_speed, follow_speed, spacing))
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue
    
    initial_guess = [
        ev_initial_params[group]['v0'],
        ev_initial_params[group]['tau'],
        ev_initial_params[group]['s0'],
        ev_initial_params[group]['delta'],
        ev_initial_params[group]['alpha'],
        ev_initial_params[group]['beta']
    ]
    
    param_bounds = [
        ev_bounds['v0'],
        ev_bounds['tau'],
        ev_bounds['s0'],
        ev_bounds['delta'],
        ev_bounds['alpha'],
        ev_bounds['beta']
    ]
    
    result = minimize(
        objective_function,
        initial_guess,
        args=(datasets, 0.02),
        method='SLSQP',
        bounds=param_bounds,
        options={'disp': True, 'maxiter': 1000}
    )
    
    ev_best_params[group] = {
        'v0': result.x[0],
        'tau': result.x[1],
        's0': result.x[2],
        'delta': result.x[3],
        'alpha': result.x[4],
        'beta': result.x[5]
    }
    
    ev_rmse_results[group] = {}
    for file_path, (time, lead_speed, follow_speed, spacing) in zip(new_groups[group], datasets):
        condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ev_best_params[group]['s0']
        idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ev_best_params[group], initial_v, initial_s, 0.02)
        speed_rmse = calculate_rmse(follow_speed, idm_speed)
        spacing_rmse = calculate_rmse(spacing, idm_spacing)
        ev_rmse_results[group][file_path] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Print best parameters and RMSEs for ICE vehicles
print("\nBest IDM Parameters and RMSEs for ICE Vehicles:")
for vehicle in ice_vehicles:
    for setting in ["Min", "Max"]:
        print(f"\nVehicle: {vehicle}, Setting: {setting}")
        print(f"v0: {ice_best_params[(vehicle, setting)]['v0']:.2f} m/s")
        print(f"tau: {ice_best_params[(vehicle, setting)]['tau']:.2f} s")
        print(f"s0: {ice_best_params[(vehicle, setting)]['s0']:.2f} m")
        print(f"delta: {ice_best_params[(vehicle, setting)]['delta']:.2f}")
        print(f"alpha: {ice_best_params[(vehicle, setting)]['alpha']:.2f} m/s²")
        print(f"beta: {ice_best_params[(vehicle, setting)]['beta']:.2f} m/s²")
        print("RMSEs:")
        for condition in ice_conditions[setting]:
            print(f" Condition {condition}:")
            print(f" Speed RMSE: {ice_rmse_results[(vehicle, setting)][condition]['speed_rmse']:.2f} m/s")
            print(f" Spacing RMSE: {ice_rmse_results[(vehicle, setting)][condition]['spacing_rmse']:.2f} m")

# Print best parameters and RMSEs for EV vehicles
print("\nBest IDM Parameters and RMSEs for EV Vehicles:")
for vehicle in list(ev_vehicles) + list(new_groups.keys()):
    print(f"\nACC Setting: {vehicle}")
    print(f"v0: {ev_best_params[vehicle]['v0']:.2f} m/s")
    print(f"tau: {ev_best_params[vehicle]['tau']:.2f} s")
    print(f"s0: {ev_best_params[vehicle]['s0']:.2f} m")
    print(f"delta: {ev_best_params[vehicle]['delta']:.2f}")
    print(f"alpha: {ev_best_params[vehicle]['alpha']:.2f} m/s²")
    print(f"beta: {ev_best_params[vehicle]['beta']:.2f} m/s²")
    print("RMSEs:")
    if vehicle in ev_vehicles:
        for condition in ev_conditions:
            if condition in ev_rmse_results[vehicle]:
                print(f" Condition {condition} km/h:")
                print(f" Speed RMSE: {ev_rmse_results[vehicle][condition]['speed_rmse']:.2f} m/s")
                print(f" Spacing RMSE: {ev_rmse_results[vehicle][condition]['spacing_rmse']:.2f} m")
    else:
        for file_path in new_groups[vehicle]:
            condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            print(f" File {os.path.basename(file_path)} (Condition {condition} km/h):")
            print(f" Speed RMSE: {ev_rmse_results[vehicle][file_path]['speed_rmse']:.2f} m/s")
            print(f" Spacing RMSE: {ev_rmse_results[vehicle][file_path]['spacing_rmse']:.2f} m")

# Plot ICE trajectories
fig_ice, axes_ice = plt.subplots(7, 16, figsize=(56, 36), sharex='col')
axes_ice = axes_ice.flatten()

# Calculate max time for ICE conditions
ice_max_times = {condition: 0 for condition in ice_conditions["Min"] + ice_conditions["Max"]}
for vehicle in ice_vehicles:
    for setting in ["Min", "Max"]:
        for condition in ice_conditions[setting]:
            csv_file = f"Veh{vehicle}_{condition}.csv"
            try:
                df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
                time = df.iloc[:, 0].values
                valid_time_mask = ~np.isnan(time)
                time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
                ice_max_times[condition] = max(ice_max_times[condition], np.max(time_shifted[~np.isnan(time_shifted)]))
            except Exception as e:
                print(f"Error reading {csv_file} for max time: {e}")

# Plot for ICE vehicles
for idx, (vehicle, condition) in enumerate([(v, c) for v in ice_vehicles for c in ice_conditions["Min"] + ice_conditions["Max"]]):
    setting = "Min" if condition.endswith("_min") else "Max"
    csv_file = f"Veh{vehicle}_{condition}.csv"
    try:
        df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
        time = df.iloc[:, 0].values
        lead_speed = df.iloc[:, 2].values
        follow_speed = df.iloc[:, 1].values
        spacing = df.iloc[:, 3].values
        
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ice_best_params[(vehicle, setting)]['s0']
        idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ice_best_params[(vehicle, setting)], initial_v, initial_s, dt=0.1)
        
        speed_rmse = ice_rmse_results[(vehicle, setting)][condition]['speed_rmse']
        spacing_rmse = ice_rmse_results[(vehicle, setting)][condition]['spacing_rmse']
        
        max_time = np.ceil(max(ice_max_times[condition], np.max(idm_time)))
        
        ax_speed = axes_ice[idx * 2]
        ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=ice_color_map[condition], linestyle='-', linewidth=1)
        ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=ice_color_map[condition], linestyle='--', linewidth=1)
        ax_speed.plot(idm_time, idm_speed, label="IDM Follow Speed", color='black', linestyle=':', linewidth=1)
        ax_speed.set_title(
            f"Veh{vehicle}_{condition} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_speed.legend(
            ["Actual Follow Speed", "Lead Speed", "IDM Follow Speed"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
        )
        ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
        ax_speed.set_xlim(0, max_time)
        ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 >= 96:
            ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
        
        ax_spacing = axes_ice[idx * 2 + 1]
        ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=ice_color_map[condition], linestyle='-', linewidth=1)
        ax_spacing.plot(idm_time, idm_spacing, label="IDM Spacing", color='black', linestyle=':', linewidth=1)
        ax_spacing.set_title(
            f"Veh{vehicle}_{condition} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_spacing.legend(
            ["Actual Spacing", "IDM Spacing"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
        )
        ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
        ax_spacing.set_xlim(0, max_time)
        ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 + 1 >= 96:
            ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")

# Save ICE plots
fig_ice.suptitle("ICE Speed and Spacing Trajectories with Optimized IDM Parameters", fontsize=14)
fig_ice.tight_layout(rect=[0, 0, 1, 0.95])
fig_ice.savefig(os.path.join(output_folder, "ice_trajectories_optimized.png"), dpi=300, bbox_inches='tight')
plt.close(fig_ice)

# Plot EV trajectories
fig_ev, axes_ev = plt.subplots(9, 8, figsize=(32, 36), sharex='col')
axes_ev = axes_ev.flatten()

# Calculate max time for EV conditions
ev_max_times = {condition: 0 for condition in ev_conditions}
for vehicle in ev_vehicles:
    for condition in ev_conditions:
        csv_file = f"{vehicle}_{condition}.csv"
        try:
            df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
            time = df['Time'].values
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            ev_max_times[condition] = max(ev_max_times[condition], np.max(time_shifted[~np.isnan(time_shifted)]))
        except Exception as e:
            print(f"Error reading {csv_file} for max time: {e}")

# Add max times for new EV groups
for group in new_groups:
    for file_path in new_groups[group]:
        try:
            condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            df = pd.read_csv(file_path)
            time = df['Time'].values
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            ev_max_times[condition] = max(ev_max_times.get(condition, 0), np.max(time_shifted[~np.isnan(time_shifted)]))
        except Exception as e:
            print(f"Error reading {file_path} for max time: {e}")

# Plot for EV vehicles
plot_idx = 0
for vehicle in ev_vehicles:
    for condition in ev_conditions:
        csv_file = f"{vehicle}_{condition}.csv"
        try:
            df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778
            follow_speed = df['Smooth Speed Follower'].values * 0.27778
            spacing = df['Spacing'].values
            
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            
            initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
            initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ev_best_params[vehicle]['s0']
            idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ev_best_params[vehicle], initial_v, initial_s, dt=0.02)
            
            speed_rmse = ev_rmse_results[vehicle][condition]['speed_rmse']
            spacing_rmse = ev_rmse_results[vehicle][condition]['spacing_rmse']
            
            max_time = np.ceil(ev_max_times[condition])
            
            ax_speed = axes_ev[plot_idx * 2]
            ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=ev_color_map[condition], linestyle='-', linewidth=1)
            ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=ev_color_map[condition], linestyle='--', linewidth=1)
            ax_speed.plot(idm_time, idm_speed, label="IDM Follow Speed", color='black', linestyle=':', linewidth=1)
            ax_speed.set_title(
                f"{vehicle}_{condition} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_speed.legend(
                ["Actual Follow Speed", "Lead Speed", "IDM Follow Speed"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
            )
            ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
            ax_speed.set_xlim(0, max_time)
            ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 >= 64:
                ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
            
            ax_spacing = axes_ev[plot_idx * 2 + 1]
            ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=ev_color_map[condition], linestyle='-', linewidth=1)
            ax_spacing.plot(idm_time, idm_spacing, label="IDM Spacing", color='black', linestyle=':', linewidth=1)
            ax_spacing.set_title(
                f"{vehicle}_{condition} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_spacing.legend(
                ["Actual Spacing", "IDM Spacing"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
            )
            ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
            ax_spacing.set_xlim(0, max_time)
            ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 + 1 >= 64:
                ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
            
            plot_idx += 1
        except Exception as e:
            print(f"Error processing {csv_file}: {e}")
            plot_idx += 1

# Plot for new EV groups
for group in new_groups:
    for file_path in new_groups[group]:
        if plot_idx * 2 >= len(axes_ev):
            print(f"Warning: Not enough subplots for {file_path}. Skipping plot.")
            continue
        try:
            condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            df = pd.read_csv(file_path)
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778
            follow_speed = df['Smooth Speed Follower'].values * 0.27778
            spacing = df['Spacing'].values
            
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            
            initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
            initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else ev_best_params[group]['s0']
            idm_time, idm_speed, idm_spacing = simulate_idm(lead_speed, time_shifted, ev_best_params[group], initial_v, initial_s, dt=0.02)
            
            speed_rmse = ev_rmse_results[group][file_path]['speed_rmse']
            spacing_rmse = ev_rmse_results[group][file_path]['spacing_rmse']
            
            max_time = np.ceil(ev_max_times[condition])
            
            ax_speed = axes_ev[plot_idx * 2]
            ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=ev_color_map[condition], linestyle='-', linewidth=1)
            ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=ev_color_map[condition], linestyle='--', linewidth=1)
            ax_speed.plot(idm_time, idm_speed, label="IDM Follow Speed", color='black', linestyle=':', linewidth=1)
            ax_speed.set_title(
                f"{group}_{os.path.basename(file_path)} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_speed.legend(
                ["Actual Follow Speed", "Lead Speed", "IDM Follow Speed"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
            )
            ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
            ax_speed.set_xlim(0, max_time)
            ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 >= 64:
                ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
            
            ax_spacing = axes_ev[plot_idx * 2 + 1]
            ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=ev_color_map[condition], linestyle='-', linewidth=1)
            ax_spacing.plot(idm_time, idm_spacing, label="IDM Spacing", color='black', linestyle=':', linewidth=1)
            ax_spacing.set_title(
                f"{group}_{os.path.basename(file_path)} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_spacing.legend(
                ["Actual Spacing", "IDM Spacing"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
            )
            ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
            ax_spacing.set_xlim(0, max_time)
            ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 + 1 >= 64:
                ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
            
            plot_idx += 1
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            plot_idx += 1

# Remove empty EV subplots
for i in range(plot_idx * 2, len(axes_ev)):
    fig_ev.delaxes(axes_ev[i])

# Save EV plots
fig_ev.suptitle("EV Speed and Spacing Trajectories with Optimized IDM Parameters", fontsize=14)
fig_ev.tight_layout(rect=[0, 0, 1, 0.95])
fig_ev.savefig(os.path.join(output_folder, "ev_trajectories_optimized.png"), dpi=300, bbox_inches='tight')
plt.close(fig_ev)

print(f"\nPlots saved to: {output_folder}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.777107646170554
            Iterations: 39
            Function evaluations: 279
            Gradient evaluations: 39
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.42838358314669
            Iterations: 24
            Function evaluations: 173
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.86602669382578
            Iterations: 16
            Function evaluations: 117
            Gradient evaluations: 16
Optimization terminated successfully    (Exit mode 0)
            Current function value: 14.272333091348202
            Iterations: 24
            Function evaluations: 172
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.055121844247953
            Iterations: 25
            Function eva